In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/Kannada-MNIST/Dig-MNIST.csv
/kaggle/input/Kannada-MNIST/train.csv
/kaggle/input/Kannada-MNIST/test.csv
/kaggle/input/Kannada-MNIST/sample_submission.csv


In [30]:
traind = pd.read_csv('/kaggle/input/Kannada-MNIST/train.csv')
testd = pd.read_csv('/kaggle/input/Kannada-MNIST/Dig-MNIST.csv')
testd.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
print(testd.loc[[11]])

    label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
11      1       0       0       0       0       0       0       0       0   

    pixel8  ...  pixel774  pixel775  pixel776  pixel777  pixel778  pixel779  \
11       0  ...         0         0         0         0         0         0   

    pixel780  pixel781  pixel782  pixel783  
11         0         0         0         0  

[1 rows x 785 columns]


In [5]:
labels = traind.iloc[:,:1]
label_one_hot = pd.get_dummies(traind['label'])
print((label_one_hot))
print(label_one_hot.shape)
#torch vectors:
labels = torch.tensor(labels.values)
label1 = torch.tensor(label_one_hot.values)

       0  1  2  3  4  5  6  7  8  9
0      1  0  0  0  0  0  0  0  0  0
1      0  1  0  0  0  0  0  0  0  0
2      0  0  1  0  0  0  0  0  0  0
3      0  0  0  1  0  0  0  0  0  0
4      0  0  0  0  1  0  0  0  0  0
...   .. .. .. .. .. .. .. .. .. ..
59995  0  0  0  0  0  1  0  0  0  0
59996  0  0  0  0  0  0  1  0  0  0
59997  0  0  0  0  0  0  0  1  0  0
59998  0  0  0  0  0  0  0  0  1  0
59999  0  0  0  0  0  0  0  0  0  1

[60000 rows x 10 columns]
(60000, 10)


In [33]:
labely = testd.iloc[:,:1]
#torch vectors:
labely = torch.tensor(labely.values)
labely = labely.reshape(10240)

In [8]:
trainx = torch.from_numpy(traind.iloc[:,1:].values).float()
print(trainx.shape)
trainx = trainx.reshape(60000,784,1)
print('trainx: ',trainx.shape)

torch.Size([60000, 784])
trainx:  torch.Size([60000, 784, 1])


In [9]:
labels = labels.reshape(60000)
print(labels.shape)

torch.Size([60000])


In [17]:
trainX = trainx
trainX = trainX.reshape(60000,784,1)
print('trainx: ',trainX.shape)
trainY = labels

x = nn.Conv1d(784,512,1)(trainX)
print(x.shape)
x = nn.MaxPool1d(1,3)(x)
print(x.shape)
x = nn.BatchNorm1d(512)(x)
print(x.shape)
x = F.relu(x)
print(x.shape)

x = nn.Conv1d(512,128,1)(x)
print(x.shape)
x = nn.MaxPool1d(1,7)(x)
print(x.shape)
x = nn.BatchNorm1d(128)(x)
print(x.shape)
x = F.relu(x)
print(x.shape)

x = nn.Conv1d(128,64,1)(x)
print(x.shape)
x = nn.AvgPool1d(1,3)(x)
print(x.shape)
x = nn.BatchNorm1d(64)(x)
print(x.shape)
x = F.relu(x)
print(x.shape)

x = nn.Conv1d(64,16,1)(x)
print(x.shape)
x = nn.AvgPool1d(1,3)(x)
print(x.shape)
x = nn.BatchNorm1d(16)(x)
print(x.shape)
x = F.relu(x)
print(x.shape)

x = nn.Conv1d(16,1,1)(x)
print(x.shape)
x = torch.sigmoid(x)
print(x.shape)

x = nn.Conv1d(1,1,1)(x)
x = x.reshape(labels.shape)
print('res:',x[1],labels[1])
print(x[0].shape,labels[0].shape)

trainx:  torch.Size([60000, 784, 1])
torch.Size([60000, 512, 1])
torch.Size([60000, 512, 1])
torch.Size([60000, 512, 1])
torch.Size([60000, 512, 1])
torch.Size([60000, 128, 1])
torch.Size([60000, 128, 1])
torch.Size([60000, 128, 1])
torch.Size([60000, 128, 1])
torch.Size([60000, 64, 1])
torch.Size([60000, 64, 1])
torch.Size([60000, 64, 1])
torch.Size([60000, 64, 1])
torch.Size([60000, 16, 1])
torch.Size([60000, 16, 1])
torch.Size([60000, 16, 1])
torch.Size([60000, 16, 1])
torch.Size([60000, 1, 1])
torch.Size([60000, 1, 1])
res: tensor([-0.6511], grad_fn=<SelectBackward>) tensor([1])
torch.Size([1]) torch.Size([1])


In [24]:
labels.shape

torch.Size([60000, 1])

In [20]:
class net(nn.Module):
    def __init__(self):
        super(net, self).__init__()
               
        self.conv1 = nn.Conv1d(784,512,1)
        self.conv2 = nn.Conv1d(512,128,1)
        self.conv3 = nn.Conv1d(128,64,1)
        self.conv4 = nn.Conv1d(64,16,1)
        self.conv5 = nn.Conv1d(16,10,1)
        self.conv6 = nn.Conv1d(1,1,1)
        
        self.max1 = nn.MaxPool1d(1,3)
        self.max2 = nn.MaxPool1d(1,7)
        self.avg1 = nn.AvgPool1d(1,3)
        self.avg2 = nn.AvgPool1d(1,7)
        
        self.batch1 = nn.BatchNorm1d(512)
        self.batch2 = nn.BatchNorm1d(128)
        self.batch3 = nn.BatchNorm1d(64)
        self.batch4 = nn.BatchNorm1d(16)
        
        self.soft = nn.LogSoftmax(dim=1)
    def forward(self, tx):
        x = self.conv1(tx)
        #print(x.shape)
        x = self.max1(x)
        #print(x.shape)
        x = self.batch1(x)
        #print(x.shape)
        x = F.relu(x)
        #print(x.shape)
        
        x = self.conv2(x)
        #print(x.shape)
        x = self.max2(x)
        #print(x.shape)
        x = self.batch2(x)
        #print(x.shape)
        x = F.relu(x)
        #print(x.shape)

        x = self.conv3(x)
        #print(x.shape)
        x = self.avg1(x)
        #print(x.shape)
        x = self.batch3(x)
        #print(x.shape)
        x = F.relu(x)
        #print(x.shape)

        x = self.conv4(x)
        #print(x.shape)
        x = self.avg2(x)
        #print(x.shape)
        x = self.batch4(x)
        #print(x.shape)
        x = F.relu(x)
        #print(x.shape)

        x = self.conv5(x)
        #print(x.shape)
        x = self.soft(x)
        #print(x.shape)

        out = x.reshape(tx.shape[0], 10)
        return out
        

In [ ]:
model = net()
crit = nn.NLLLoss()
opt = optim.Adam(model.parameters(),lr = 0.09)#0.22

for i in range(10):
    out = model(trainx)
    loss = crit(out,labels)
    print(i,loss.item())
    loss.backward()
    opt.step()
    opt.zero_grad()

0 2.381065607070923
1 2.6660096645355225
2 2.106387138366699
3 1.8797755241394043
4 1.6299774646759033
5 1.3230628967285156
6 1.0911966562271118
7 0.8781529068946838
8 0.7108678817749023


In [36]:
print(crit(model(testx),labely))

tensor(1.2713, grad_fn=<NllLossBackward>)


In [34]:
testx = torch.from_numpy(testd.iloc[:,1:].values).float()
print(testx.shape)
testx = testx.reshape(10240,784,1)
print('testx: ',testx.shape)


torch.Size([10240, 784])
testx:  torch.Size([10240, 784, 1])


In [35]:
print(testx.shape)
print(labely.shape)
print(labely)

torch.Size([10240, 784, 1])
torch.Size([10240])
tensor([0, 1, 2,  ..., 7, 8, 9])
